In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['TEC', 'TECs']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('thymic epithelial cells', 208),
 ('tubular epithelial cells', 120),
 ('transcription elongation complex', 42),
 ('tumor endothelial cells', 42),
 ('thyroid epithelial cells', 15),
 ('triethyl citrate', 14),
 ('total electron content', 10),
 ('triethylcholine', 9),
 ('thymic epithelial cells line', 8),
 ('ternary elongation complex', 7),
 ('total erythrocyte count', 6),
 ('tumor derived endothelial cells', 6),
 ('transcription elongation complexes', 6),
 ('thermal expansion coefficient', 5),
 ('tracheal epithelial cells', 5),
 ('tumor associated endothelial cells', 5),
 ('tumour endothelial cells', 4),
 ('total eosinophil count', 3),
 ('thromboembolic complications', 3),
 ('tissue engineered constructs', 3),
 ('tumor ecs', 3),
 ('tubule epithelial cells', 3)]

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'MESH:D002477': 'Cells',
 'MESH:C023001': 'ethyl citrate',
 'MESH:C100239': 'triethylcholine',
 'MESH:D004906': 'Erythrocyte Count',
 'EFO:0004842': 'eosinophil count',
 'NCIT:C33771': 'Thymic Epithelial Cell',
 'transcription elongation complex': 'transcription elongation complex',
 'NCIT:C61147': 'Renal Tubular Epithelial Cell',
 'NCIT:C37088': 'Neoplastic Endothelial Cell',
 'MESH:D000072637': 'Thyroid Epithelial Cells',
 'PUBCHEM:6506': 'Triethyl citrate',
 'HGNC:11719': 'TEC'}

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-11 04:22:20] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'ternary elongation complex': 'transcription elongation complex',
  'thermal expansion coefficient': 'ungrounded',
  'thromboembolic complications': 'ungrounded',
  'thymic epithelial cells': 'NCIT:C33771',
  'thymic epithelial cells line': 'NCIT:C33771',
  'thyroid epithelial cells': 'MESH:D000072637',
  'tissue engineered constructs': 'ungrounded',
  'total electron content': 'ungrounded',
  'total eosinophil count': 'EFO:0004842',
  'total erythrocyte count': 'MESH:D004906',
  'tracheal epithelial cells': 'ungrounded',
  'transcription elongation complex': 'transcription elongation complex',
  'transcription elongation complexes': 'transcription elongation complex',
  'triethyl citrate': 'PUBCHEM:6506',
  'triethylcholine': 'MESH:C100239',
  'tubular epithelial cells': 'NCIT:C61147',
  'tubule epithelial cells': 'NCIT:C61147',
  'tumor associated endothelial cells': 'NCIT:C37088',
  'tumor derived endothelial cells': 'NCIT:C37088',
  'tumor ecs': 'NCIT:C37088',
  'tumor endothelia

In [11]:
grounding_map, names, pos_labels = [{'ternary elongation complex': 'transcription elongation complex',
  'thermal expansion coefficient': 'ungrounded',
  'thromboembolic complications': 'ungrounded',
  'thymic epithelial cells': 'NCIT:C33771',
  'thymic epithelial cells line': 'NCIT:C33771',
  'thyroid epithelial cells': 'MESH:D000072637',
  'tissue engineered constructs': 'ungrounded',
  'total electron content': 'ungrounded',
  'total eosinophil count': 'EFO:0004842',
  'total erythrocyte count': 'MESH:D004906',
  'tracheal epithelial cells': 'ungrounded',
  'transcription elongation complex': 'transcription elongation complex',
  'transcription elongation complexes': 'transcription elongation complex',
  'triethyl citrate': 'PUBCHEM:6506',
  'triethylcholine': 'MESH:C100239',
  'tubular epithelial cells': 'NCIT:C61147',
  'tubule epithelial cells': 'NCIT:C61147',
  'tumor associated endothelial cells': 'NCIT:C37088',
  'tumor derived endothelial cells': 'NCIT:C37088',
  'tumor ecs': 'NCIT:C37088',
  'tumor endothelial cells': 'NCIT:C37088',
  'tumour endothelial cells': 'NCIT:C37088'},
 {'transcription elongation complex': 'transcription elongation complex',
  'NCIT:C33771': 'Thymic Epithelial Cell',
  'MESH:D000072637': 'Thyroid Epithelial Cells',
  'EFO:0004842': 'eosinophil count',
  'MESH:D004906': 'Erythrocyte Count',
  'PUBCHEM:6506': 'Triethyl citrate',
  'MESH:C100239': 'triethylcholine',
  'NCIT:C61147': 'Renal Tubular Epithelial Cell',
  'NCIT:C37088': 'Neoplastic Endothelial Cell'},
 ['MESH:D000072637',
  'NCIT:C33771',
  'NCIT:C37088',
  'NCIT:C61147',
  'PUBCHEM:6506']]

In [12]:
excluded_longforms = []

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {'HGNC:11719': ['TEC', ['TEC']]}

In [15]:
unambiguous_agent_texts = {}

In [16]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [17]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [18]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [19]:
intersection1

[('HGNC:11719', 'HGNC:11719', 564)]

In [20]:
intersection2

[('NCIT:C61147', 'HGNC:11719', 0),
 ('ungrounded', 'HGNC:11719', 0),
 ('NCIT:C33771', 'HGNC:11719', 9),
 ('MESH:D000072637', 'HGNC:11719', 0),
 ('MESH:C100239', 'HGNC:11719', 0),
 ('EFO:0004842', 'HGNC:11719', 0),
 ('MESH:D004906', 'HGNC:11719', 0),
 ('PUBCHEM:6506', 'HGNC:11719', 0),
 ('transcription elongation complex', 'HGNC:11719', 0),
 ('NCIT:C37088', 'HGNC:11719', 0)]

In [21]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [22]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [23]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-11 04:28:04] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-11 04:28:16] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9150433814494592 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [24]:
classifier.stats

{'label_distribution': {'NCIT:C61147': 117,
  'ungrounded': 15,
  'NCIT:C33771': 211,
  'MESH:D000072637': 13,
  'MESH:C100239': 6,
  'EFO:0004842': 3,
  'MESH:D004906': 4,
  'PUBCHEM:6506': 10,
  'transcription elongation complex': 51,
  'NCIT:C37088': 42,
  'HGNC:11719': 64},
 'f1': {'mean': 0.915043, 'std': 0.017486},
 'precision': {'mean': 0.905184, 'std': 0.00861},
 'recall': {'mean': 0.936642, 'std': 0.018432},
 'ungrounded': {'f1': {'mean': 0.08, 'std': 0.16},
  'pr': {'mean': 0.066667, 'std': 0.133333},
  'rc': {'mean': 0.1, 'std': 0.2}},
 'MESH:C100239': {'f1': {'mean': 1.0, 'std': 0.0},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D000072637': {'f1': {'mean': 0.533333, 'std': 0.323179},
  'pr': {'mean': 0.466667, 'std': 0.339935},
  'rc': {'mean': 0.733333, 'std': 0.38873}},
 'transcription elongation complex': {'f1': {'mean': 1.0, 'std': 0.0},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D004906': {'f1': {'mea

In [25]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [26]:
disamb.dump(model_name, results_path)

In [27]:
print(disamb.info())

Disambiguation model for TEC, and TECs

Produces the disambiguations:
	Erythrocyte Count	MESH:D004906
	Neoplastic Endothelial Cell*	NCIT:C37088
	Renal Tubular Epithelial Cell*	NCIT:C61147
	TEC*	HGNC:11719
	Thymic Epithelial Cell*	NCIT:C33771
	Thyroid Epithelial Cells*	MESH:D000072637
	Triethyl citrate*	PUBCHEM:6506
	eosinophil count	EFO:0004842
	transcription elongation complex	transcription elongation complex
	triethylcholine	MESH:C100239

Class level metrics:
--------------------
Grounding                       	Count	F1     
          Thymic Epithelial Cell*	211	0.93566
   Renal Tubular Epithelial Cell*	117	0.94145
                             TEC*	 64	0.85563
transcription elongation complex	 51	    1.0
     Neoplastic Endothelial Cell*	 42	0.93824
                      Ungrounded	 15	   0.08
        Thyroid Epithelial Cells*	 13	0.53333
                Triethyl citrate*	 10	0.93333
                 triethylcholine	  6	    1.0
               Erythrocyte Count	  4	    0.6
          

In [28]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1